In [1]:
import re
import os
import json
import random
from tqdm.notebook import tqdm
from dotenv import load_dotenv
from huggingface_hub import login
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer
from datasets import load_dataset, Dataset, DatasetDict
import chromadb
from sklearn.manifold import TSNE
import plotly.graph_objects as go

In [2]:
load_dotenv(override=True)

True

In [3]:
hf_token = os.getenv("HF_TOKEN")

In [4]:
login(hf_token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [5]:
from items import Item

In [6]:
with open("train.pkl","rb") as file:
    train = pickle.load(file)

In [7]:
with open("test.pkl","rb") as file:
    test = pickle.load(file)

In [8]:
print(train[0].test_prompt())

How much does this cost to the nearest dollar?

Delphi FG0166 Fuel Pump Module
Delphi brings 80 years of OE Heritage into each Delphi pump, ensuring quality and fitment for each Delphi part. Part is validated, tested and matched to the right vehicle application Delphi brings 80 years of OE Heritage into each Delphi assembly, ensuring quality and fitment for each Delphi part Always be sure to check and clean fuel tank to avoid unnecessary returns Rigorous OE-testing ensures the pump can withstand extreme temperatures Brand Delphi, Fit Type Vehicle Specific Fit, Dimensions LxWxH 19.7 x 7.7 x 5.1 inches, Weight 2.2 Pounds, Auto Part Position Unknown, Operation Mode Mechanical, Manufacturer Delphi, Model FUEL PUMP, Dimensions 19.7

Price is $


In [9]:
DB = "products_vectorstore"

In [10]:
# !pip install chromadb==0.5.0

In [11]:
client = chromadb.PersistentClient(path=DB)

In [12]:
collection_name = "products"

existing_collection_names = client.list_collections()
if collection_name in existing_collection_names:
    client.delete_collection(collection_name)
    print(f"Deleted collection name: {collection_name}")
collection = client.create_collection(collection_name)

In [13]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [14]:
model.encode(["Hello There Obi Wan Kenobi"])[0]

array([-5.20291142e-02,  8.64912421e-02,  1.52621055e-02,  6.82936329e-03,
       -5.85784875e-02, -3.50730829e-02,  8.33703056e-02,  1.01704206e-02,
        3.77711281e-02,  2.21600421e-02,  2.68390030e-02, -7.37570673e-02,
       -4.87001762e-02,  1.18465768e-02,  5.70595078e-02, -6.19011931e-03,
        6.88730506e-03, -1.32260183e-02, -1.34120494e-01,  1.14312042e-02,
        3.37507650e-02,  3.22072655e-02, -1.69004202e-02, -2.96870042e-02,
       -1.16879039e-01,  3.18301320e-02,  6.00073263e-02,  3.71701866e-02,
        4.54801247e-02,  4.81947372e-03,  4.80717234e-02,  5.91164343e-02,
       -1.88030377e-02, -1.86225567e-02,  3.14410850e-02, -2.33333348e-03,
       -5.26830815e-02, -3.36500853e-02,  1.32554859e-01, -6.88477308e-02,
        7.90474489e-02, -5.06058745e-02,  1.97300576e-02, -6.23440668e-02,
        8.78800824e-02, -9.63441432e-02, -2.66774818e-02, -1.42193837e-02,
        1.51364699e-01, -3.69481705e-02, -4.05185744e-02, -5.28324768e-02,
        3.55444290e-02,  

In [15]:
def cosine_similarity(a,b):
    return np.dot(a,b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [16]:
def how_similar(text1, text2):
    vector1, vector2 = model.encode([text1, text2])
    similarity = cosine_similarity(vector1, vector2)
    print(f"Similarity between {text1} and {text2} is {similarity*100:.1f}%")

In [17]:
how_similar("HI there", "general kenobi")

Similarity between HI there and general kenobi is 39.4%


In [18]:
how_similar("do this work", "Ok I will perform the task")

Similarity between do this work and Ok I will perform the task is 29.2%


In [19]:
how_similar("Java", "C++")

Similarity between Java and C++ is 50.7%


In [20]:
how_similar("Rust","Erlang")

Similarity between Rust and Erlang is 29.7%


In [21]:
def description(item):
    text = item.prompt.replace("How much does this cost to the nearest dollar?\n\n", "")
    return text.split("\n\nPrice is $")[0]

In [22]:
description(train[0])

'Delphi FG0166 Fuel Pump Module\nDelphi brings 80 years of OE Heritage into each Delphi pump, ensuring quality and fitment for each Delphi part. Part is validated, tested and matched to the right vehicle application Delphi brings 80 years of OE Heritage into each Delphi assembly, ensuring quality and fitment for each Delphi part Always be sure to check and clean fuel tank to avoid unnecessary returns Rigorous OE-testing ensures the pump can withstand extreme temperatures Brand Delphi, Fit Type Vehicle Specific Fit, Dimensions LxWxH 19.7 x 7.7 x 5.1 inches, Weight 2.2 Pounds, Auto Part Position Unknown, Operation Mode Mechanical, Manufacturer Delphi, Model FUEL PUMP, Dimensions 19.7'

In [23]:
NUMBER_OF_DOCUMENTS = 20000
for i in tqdm(range(0, NUMBER_OF_DOCUMENTS,1000)):
    documents = [description(item) for item in train[i:i+1000]]
    vectors = model.encode(documents).astype(float).tolist()
    metadatas = [{"category":item.category,"price": item.price} for item in train[i:i+1000]]
    ids = [f"doc__{j}" for j in range(i,i+len(documents))]
    collection.add(
        ids=ids,
        documents=documents,
        metadatas=metadatas,
        embeddings=vectors
    )

  0%|          | 0/20 [00:00<?, ?it/s]